In [82]:
# Imports

from jointformer.configs.dataset import DatasetConfig
from jointformer.utils.datasets.auto import AutoDataset

# import os
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import sklearn
# from typing import List

# from rdkit import Chem
# from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
# from simpletransformers.classification import ClassificationModel, ClassificationArgs

# # import MoleculeNet loaders from DeepChem
# from deepchem.molnet import load_bbbp, load_clearance, load_clintox, load_delaney, load_hiv, load_qm7, load_tox21

# # import MoleculeNet dataloder from bert-loves-chemistry fork
# # %cd /content/bert-loves-chemistry
# # from chemberta.utils.molnet_dataloader import load_molnet_dataset, write_molnet_dataset_for_chemprop
# from jointformer.models.chemberta.chemberta.utils.molnet_dataloader import load_molnet_dataset, write_molnet_dataset_for_chemprop

# https://github.com/SylwiaNowakowska/LLM_Fine_Tuning_Molecular_Properties/blob/main/01_ChemBERTa2_HIV_FineTuning.ipynb

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
# Configs

DATA_DIR = '/home/adamizdebski/files/jointformer/data'
PATH_TO_DATASET_CONFIG = '/home/adamizdebski/projects/jointformer/configs/datasets/molecule_net/esol'

In [90]:


dataset_config = DatasetConfig.from_config_file(PATH_TO_DATASET_CONFIG)
dataset = AutoDataset.from_config(dataset_config, data_dir=DATA_DIR, split='train', seed=0, num_samples=100)
train_df = dataset.get_data_frame()


In [8]:
# set the logging directories
project_name = "ChemBERTA_HIV"
output_path = '/content/drive/MyDrive/COLAB_NOTEBOOKS/LLM_Fine_Tuning_Molecular_Properties_output'
model_name = 'model_36'

model_folder = os.path.join(output_path, model_name)

evaluation_folder = os.path.join(output_path, model_name + '_evaluation')
if not os.path.exists(evaluation_folder):
    os.makedirs(evaluation_folder)

# set the parameters
EPOCHS = 200
BATCH_SIZE = 256
patience = 15
optimizer = "AdamW"
learning_rate = 0.00001
manual_seed = 112
print(model_folder)

classification_args = {'evaluate_each_epoch': True,
                       'evaluate_during_training_verbose': True,
                       'evaluate_during_training' : True,
                       'best_model_dir' : model_folder,
                       'no_save': False,
                       'save_eval_checkpoints': False,
                       'save_model_every_epoch': False,
                       'save_best_model' : True,
                       'save_steps': -1,
                       'num_train_epochs': EPOCHS,
                       'use_early_stopping': True,
                       'early_stopping_patience': patience,
                       'early_stopping_delta': 0.001,
                       'early_stopping_metrics': 'eval_loss',
                       'early_stopping_metrics_minimize': True,
                       'early_stopping_consider_epochs' : True,
                       'fp16' : False,
                       'optimizer' : optimizer,
                       'adam_betas' : (0.95, 0.999),
                       'learning_rate' : learning_rate,
                       'manual_seed': manual_seed,
                       'train_batch_size' : BATCH_SIZE,
                       'eval_batch_size' : BATCH_SIZE,
                       'logging_steps' : len(train_df) / BATCH_SIZE,
                       'auto_weights': True, # change to true
                       'wandb_project': project_name,
                       'wandb_kwargs': wandb_kwargs}

model = ClassificationModel('roberta', 'DeepChem/ChemBERTa-77M-MLM', args=classification_args)

PermissionError: [Errno 13] Permission denied: '/content'